In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
pip install split-folders

In [0]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input
from keras.preprocessing import image
import split_folders
import glob
import os
from PIL import Image
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [0]:
!unzip '/content/drive/My Drive/VR/Dress Recognition/cropped_images.zip'

Streaming output truncated to the last 5000 lines.
  inflating: cropped_images/saree/1048.png  
  inflating: cropped_images/saree/1049.png  
  inflating: cropped_images/saree/105.png  
  inflating: cropped_images/saree/527.png  
  inflating: cropped_images/saree/532.png  
  inflating: cropped_images/saree/535.png  
  inflating: cropped_images/saree/537.png  
  inflating: cropped_images/saree/1052.png  
  inflating: cropped_images/saree/1053.png  
  inflating: cropped_images/saree/1987.png  
  inflating: cropped_images/saree/1988.png  
  inflating: cropped_images/saree/1989.png  
  inflating: cropped_images/saree/199.png  
  inflating: cropped_images/saree/1990.png  
  inflating: cropped_images/saree/1991.png  
  inflating: cropped_images/saree/1992.png  
  inflating: cropped_images/saree/1993.png  
  inflating: cropped_images/saree/1994.png  
  inflating: cropped_images/saree/1995.png  
  inflating: cropped_images/saree/1996.png  
  inflating: cropped_images/saree/1997.png  
  inflatin

In [0]:
split_folders.ratio('cropped_images', output="output", seed=1337, ratio= (.8,.2)) # default values

Copying files: 6383 files [00:02, 3042.54 files/s]


In [0]:
train_path = '/content/output/train/'
valid_path = '/content/output/val/'

In [0]:
# Image size should be [224,224]
# Iterate through each color folder

def resize_images(image_dir):
  image_types = ["kurti","none","saree","shirt","tshirt"]
  for im_type in image_types:
    # Iterate through each image file in each image_type folder
    #  glob reads in any image with the extension "image_dir/im_type/*"
    for file in glob.glob(os.path.join(image_dir, im_type, "*")):
        im = Image.open(file)
        f, e = os.path.splitext(file)
        imResize = im.resize((224,224), Image.ANTIALIAS)
        imResize.save(f + '.png', 'PNG', quality=90)

resize_images('/content/output/val')
resize_images('/content/output/train')

In [0]:
image_size = [224,224]
num_classes = 5
# add preprocessing layer to the front of ResNet
resnet = ResNet50(input_shape=image_size + [3], weights='imagenet', include_top=False)

# don't train existing weights
for layer in resnet.layers:
  layer.trainable = False

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 3s 0us/step


In [0]:
# our layers - you can add more if you want
x = Flatten()(resnet.output)
# x = Dense(1000, activation='relu')(x)
prediction = Dense(num_classes, activation='softmax')(x)

# create a model object
model = Model(inputs=resnet.input, outputs=prediction)

# view the structure of the model
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

In [0]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [0]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

val_datagen = ImageDataGenerator(rescale = 1./255)

In [0]:
training_set = train_datagen.flow_from_directory('/content/output/train/',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 5105 images belonging to 5 classes.


In [0]:
validation_set = val_datagen.flow_from_directory('/content/output/val/',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 1278 images belonging to 5 classes.


In [0]:
# fit the model
r = model.fit_generator(
  training_set,
  validation_data=validation_set,
  epochs=10,
  steps_per_epoch=len(training_set),
  validation_steps=len(validation_set)
)

Epoch 1/10
160/160 [==============================] - 80s 500ms/step - loss: 3.9240 - accuracy: 0.7459 - val_loss: 9.8968 - val_accuracy: 0.2457
Epoch 2/10
160/160 [==============================] - 68s 425ms/step - loss: 2.6575 - accuracy: 0.8533 - val_loss: 17.0408 - val_accuracy: 0.1901
Epoch 3/10
160/160 [==============================] - 69s 431ms/step - loss: 2.0128 - accuracy: 0.8925 - val_loss: 23.8024 - val_accuracy: 0.1964
Epoch 4/10
160/160 [==============================] - 69s 430ms/step - loss: 2.1814 - accuracy: 0.8927 - val_loss: 33.4269 - val_accuracy: 0.1878
Epoch 5/10
160/160 [==============================] - 68s 426ms/step - loss: 1.6067 - accuracy: 0.9216 - val_loss: 30.4091 - val_accuracy: 0.1878
Epoch 6/10
160/160 [==============================] - 68s 425ms/step - loss: 1.4271 - accuracy: 0.9303 - val_loss: 37.6189 - val_accuracy: 0.1878
Epoch 7/10
160/160 [==============================] - 68s 424ms/step - loss: 1.5217 - accuracy: 0.9352 - val_loss: 30.9970 - 

In [0]:
# loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

In [0]:
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [0]:
# model.save_weights('pre_trained_resnet.h5')